<a href="https://colab.research.google.com/github/TheBlackRus/Manning_LP_deploy_ml_as_microservice/blob/main/deploy_ml_as_micro_score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [ ]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [ ]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)

### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [ ]:
# %pip install algorithmia
import Algorithmia
client = Algorithmia.client('YOUR_KEY')
algo = client.algo('nlp/SentimentAnalysis/1.0.5')
algo.set_options(timeout=300) # optional
def score_review_algortihma(review):

  input = {
    "document": review
  }

  print(algo.pipe(input).result)
  score = algo.pipe(input).result[0]['sentiment']
  return score

In [ ]:
print(reviews_pos[0])
score_review_algortihma("I hate this")

films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before . 
for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen . 
to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . 
the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . 
in other words , don't dismiss this film because of its source . 
if you can get past the whole comic book thing , you might find another stumbling block in from hell's directors , albert and allen hughes . 
getting the hughes brothers to direct this seems almost as 

-0.5719

In [ ]:
from google.cloud import language_v1
def score_review_google(review):
  """
  Analyzing Sentiment in a String

  Args:
    text_content The text content to analyze
  """

  client = language_v1.LanguageServiceClient()

  # text_content = 'I am so happy and joyful.'

  # Available types: PLAIN_TEXT, HTML
  type_ = language_v1.Document.Type.PLAIN_TEXT

  # Optional. If not specified, the language is automatically detected.
  # For list of supported languages:
  # https://cloud.google.com/natural-language/docs/languages
  language = "en"
  document = {"content": text_content, "type_": type_, "language": language}

  # Available values: NONE, UTF8, UTF16, UTF32
  encoding_type = language_v1.EncodingType.UTF8

  response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
  # Get overall sentiment of the input document
  print(u"Document sentiment score: {}".format(response.document_sentiment.score))
  print(
      u"Document sentiment magnitude: {}".format(
          response.document_sentiment.magnitude
      )
  )
  # Get sentiment for all sentences in the document
  for sentence in response.sentences:
      print(u"Sentence text: {}".format(sentence.text.content))
      print(u"Sentence sentiment score: {}".format(sentence.sentiment.score))
      print(u"Sentence sentiment magnitude: {}".format(sentence.sentiment.magnitude))

  # Get the language of the text, which will be the same as
  # the language specified in the request or, if not specified,
  # the automatically-detected language.
  print(u"Language of the text: {}".format(response.language))
  return response.document_sentiment.magnitude


In [ ]:
score_review_func = score_review_algortihma#score_review_google

In [ ]:
def score_review(review):
    # TBD: call the service and return 'pos' or 'neg'
    res = score_review_func(review)
    if res >= 0:
      return 'pos'
    else:
      return 'neg'

### Score each review

Now, we can use the function you defined to score each of the reviews.

#### *Note on Testing*

While most of the services listed have free tiers they may be limited to a few thousand requests per week or month, depending on the service. On some platforms you may be billed after reaching that limit. For this reason it is recommended to first test on a smaller set of the reviews, `subset_pos` and `subset_neg`. Once you're happy with your code swap those subsets for the full review sets `reviews_pos` and `reviews_neg`.

In [ ]:
# Create 2 smaller subsets for testing
subset_pos = reviews_pos[:10]
subset_neg = reviews_neg[:10]

results_pos = []
# When comfortable with results switch `subset_pos` to reviews_post`
for review in subset_pos:
    result = score_review(review)
    results_pos.append(result)

results_neg = []
# When comfortable with results switch `subset_neg` to reviews_neg`
for review in subset_neg:
    result = score_review(review)
    results_neg.append(result)

[{'document': 'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers to di

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [ ]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 80.0% correct
Negative reviews: 50.0% correct
Overall accuracy: 65.0% correct
